In [1]:
import pandas as pd
import numpy as np
from typing import Tuple, List
from sklearn.preprocessing import RobustScaler
import joblib

class CreditScoringPreprocessor:
    """
    Класс для предобработки данных с сохранением всех параметров
    """
    
    def __init__(self, RANDOM_STATE: int = 42):
        self.RANDOM_STATE = RANDOM_STATE
        
        # Фиксированные маппинги
        self.rating_order = {'А': 1, 'Б': 2, 'В': 3, 'Г': 4, 'Д': 5, 'Е': 6, 'Ж': 7}
        self.experience_order = {
            '< 1 года': 0.5, '1 год': 1, '2 года': 2, '3 года': 3, 
            '4 года': 4, '5 лет': 5, '6 лет': 6, '7 лет': 7, 
            '8 лет': 8, '9 лет': 9, '10+ лет': 10
        }
        self.binary_mapping = {
            'Да': 1, 'Нет': 0, 'Под вопросом': 0.5,
            'True': 1, 'False': 0
        }
        
        # Для сохранения параметров
        self.medians_ = {}
        self.freq_encoders_ = {}
        self.scaler_ = None
        self.onehot_categories_ = {}
        self.feature_names_ = None
        
        # Признаки для удаления
        self.useless_cols = [
            'дата_следующей_выплаты',
            'кредитный_баланс_по_возоб_счетам',
            'совокупный_статус_подтверждения_доходов_заемщиков',
            'совокупный_пдн_заемщиков',
            'совокупный_доход_заемщиков',
        ]
        
        self.constant_cols = [
            'платежный_график',
            'особая_ситуация',
        ]
        
        self.binary_cols = ['пос_стоп_фактор', 'юридический_статус']
        self.onehot_cols = [
            'владение_жильем', 'подтвержден_ли_доход',
            'первоначальный_статус_займа', 'тип_займа',
            'тип_предоставления_кредита'
        ]
        self.freq_cols = ['профессия_заемщика', 'допрейтинг', 'регион']
    
    def fit(self, train_df: pd.DataFrame, target_col: str = 'итоговый_статус_займа'):
        """
        Обучает препроцессор на train данных и сохраняет все параметры
        """
        print("="*60)
        print("ОБУЧЕНИЕ ПРЕПРОЦЕССОРА")
        print("="*60)
        
        train_df_processed = train_df.copy()
        
        # Сохраняем ID и target
        self.train_ids_ = train_df_processed['id'].copy()
        if target_col in train_df_processed.columns:
            self.y_ = train_df_processed[target_col].copy()
        
        print(f"Начальный размер: {train_df_processed.shape}")
        
        # 1. Удаление бесполезных признаков
        cols_to_drop = [col for col in self.useless_cols + self.constant_cols 
                       if col in train_df_processed.columns]
        train_df_processed = train_df_processed.drop(columns=cols_to_drop, errors='ignore')
        print(f"✓ Удалено {len(cols_to_drop)} признаков")
        
        # 2. Обработка дат
        if 'дата_первого_займа' in train_df_processed.columns:
            train_df_processed['дата_первого_займа'] = pd.to_datetime(
                train_df_processed['дата_первого_займа'], format='%m-%Y', errors='coerce'
            )
            current_date = pd.Timestamp('2026-01-01')
            train_df_processed['стаж_кредитной_истории_мес'] = (
                (current_date - train_df_processed['дата_первого_займа']).dt.days / 30
            ).fillna(0)
            train_df_processed['стаж_кредитной_истории_мес'] = train_df_processed['стаж_кредитной_истории_мес'].clip(0, 600)
            train_df_processed = train_df_processed.drop(columns=['дата_первого_займа'])
            print("✓ Создан признак: стаж_кредитной_истории_мес")
        
        # 3. Обработка сроков займа
        if 'срок_займа' in train_df_processed.columns:
            train_df_processed['срок_займа_мес'] = train_df_processed['срок_займа'].str.extract('(\d+)').astype(float)
            train_df_processed['срок_займа_мес'] = train_df_processed['срок_займа_мес'] * 12
            train_df_processed = train_df_processed.drop(columns=['срок_займа'])
            print("✓ Создан признак: срок_займа_мес")
        
        # 4. Кодирование рейтингов
        if 'рейтинг' in train_df_processed.columns:
            train_df_processed['рейтинг_encoded'] = train_df_processed['рейтинг'].map(self.rating_order).fillna(0)
            train_df_processed = train_df_processed.drop(columns=['рейтинг'])
            print("✓ Закодирован рейтинг")
        
        # 5. Стаж работы
        if 'стаж' in train_df_processed.columns:
            train_df_processed['стаж_encoded'] = train_df_processed['стаж'].map(self.experience_order)
            self.medians_['стаж_encoded'] = train_df_processed['стаж_encoded'].median()
            train_df_processed['стаж_encoded'] = train_df_processed['стаж_encoded'].fillna(self.medians_['стаж_encoded'])
            train_df_processed = train_df_processed.drop(columns=['стаж'])
            print("✓ Закодирован стаж")
        
        # 6. Бинарные признаки
        for col in self.binary_cols:
            if col in train_df_processed.columns:
                train_df_processed[col] = train_df_processed[col].map(self.binary_mapping).fillna(0)
                print(f"✓ Закодирован {col}")
        
        # 7. One-Hot Encoding (сохраняем категории)
        for col in self.onehot_cols:
            if col in train_df_processed.columns:
                # Заполняем пропуски
                train_df_processed[col] = train_df_processed[col].fillna('MISSING')
                
                # Сохраняем уникальные категории
                self.onehot_categories_[col] = train_df_processed[col].unique().tolist()
                print(f"✓ Сохранены категории для {col}: {len(self.onehot_categories_[col])} значений")
        
        # 8. Frequency Encoding (сохраняем частоты)
        for col in self.freq_cols:
            if col in train_df_processed.columns:
                train_df_processed[col] = train_df_processed[col].fillna('MISSING')
                freq = train_df_processed[col].value_counts(normalize=True)
                self.freq_encoders_[col] = freq
                print(f"✓ Сохранены частоты для {col}")
        
        # 9. Цель займа (группировка)
        if 'цель_займа' in train_df_processed.columns:
            purpose_groups = {
                'консолидация_долга': ['консолидация_долга'],
                'кредитная_карта': ['кредитная_карта'],
                'жилье': ['улучшение_жилищных_условий', 'дом'],
                'бизнес': ['мелкий_бизнес'],
                'авто': ['автомобиль'],
                'образование': ['образование'],
                'лечение': ['лечение'],
                'переезд': ['переезд'],
                'отпуск': ['отпуск'],
                'другое': ['другое', 'крупная_покупка', 'возобновляемая_энергия', 'свадьба']
            }
            
            purpose_to_group = {}
            for group, purposes in purpose_groups.items():
                for purpose in purposes:
                    purpose_to_group[purpose] = group
            
            train_df_processed['цель_займа'] = train_df_processed['цель_займа'].fillna('другое')
            train_df_processed['цель_займа_группа'] = train_df_processed['цель_займа'].map(purpose_to_group)
            train_df_processed.loc[train_df_processed['цель_займа_группа'].isna(), 'цель_займа_группа'] = 'другое'
            
            # Сохраняем уникальные группы
            self.onehot_categories_['цель_займа_группа'] = train_df_processed['цель_займа_группа'].unique().tolist()
            print("✓ Обработана цель_займа")
        
        # 10. Обработка пени_за_дефолт
        if 'пени_за_дефолт' in train_df_processed.columns:
            train_df_processed['пени_за_дефолт'] = train_df_processed['пени_за_дефолт'].map({'True': 1, 'False': 0})
            self.medians_['пени_за_дефолт'] = train_df_processed['пени_за_дефолт'].median()
            print("✓ Обработано пени_за_дефолт")
        
        # 11. Финансовые соотношения
        if all(col in train_df_processed.columns for col in ['аннуитет', 'годовой_доход']):
            train_df_processed['годовой_доход_safe'] = train_df_processed['годовой_доход'].replace(0, 1)
            train_df_processed['аннуитет_к_доходу'] = train_df_processed['аннуитет'] * 12 / train_df_processed['годовой_доход_safe']
            train_df_processed = train_df_processed.drop(columns=['годовой_доход_safe'])
            print("✓ Создан: аннуитет_к_доходу")
        
        if all(col in train_df_processed.columns for col in ['пдн', 'годовой_доход']):
            train_df_processed['годовой_доход_safe'] = train_df_processed['годовой_доход'].replace(0, 1)
            train_df_processed['пдн_от_дохода'] = train_df_processed['пдн'] / train_df_processed['годовой_доход_safe']
            train_df_processed = train_df_processed.drop(columns=['годовой_доход_safe'])
            print("✓ Создан: пдн_от_дохода")
        
        if all(col in train_df_processed.columns for col in ['сумма_займа', 'годовой_доход']):
            train_df_processed['годовой_доход_safe'] = train_df_processed['годовой_доход'].replace(0, 1)
            train_df_processed['заем_к_доходу'] = train_df_processed['сумма_займа'] / train_df_processed['годовой_доход_safe']
            train_df_processed = train_df_processed.drop(columns=['годовой_доход_safe'])
            print("✓ Создан: заем_к_доходу")
        
        # 12. Заполнение пропусков в числовых признаках
        numeric_cols = train_df_processed.select_dtypes(include=[np.number]).columns.tolist()
        for col in numeric_cols:
            if col in train_df_processed.columns and train_df_processed[col].isnull().any() and col != target_col:
                self.medians_[col] = train_df_processed[col].median()
                train_df_processed[col] = train_df_processed[col].fillna(self.medians_[col])
        
        # 13. Масштабирование
        self.scaler_ = RobustScaler()
        numeric_cols_to_scale = train_df_processed.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_cols_to_scale:
            train_df_processed[numeric_cols_to_scale] = self.scaler_.fit_transform(train_df_processed[numeric_cols_to_scale])
            print(f"✓ Обучен scaler на {len(numeric_cols_to_scale)} признаках")
        
        # 14. Удаление константных признаков
        constant_cols_final = [col for col in train_df_processed.columns if train_df_processed[col].nunique() == 1]
        if constant_cols_final:
            train_df_processed = train_df_processed.drop(columns=constant_cols_final, errors='ignore')
            print(f"✓ Удалено {len(constant_cols_final)} константных признаков")
        
        # Сохраняем имена фичей
        self.feature_names_ = train_df_processed.columns.tolist()
        
        print("\n" + "="*60)
        print("ПРЕПРОЦЕССОР ОБУЧЕН!")
        print(f"Количество признаков: {len(self.feature_names_)}")
        print("="*60)
        
        return self
    
    def transform(self, df: pd.DataFrame, target_col: str = None):
        """
        Применяет предобработку к новым данным с использованием сохраненных параметров
        """
        print("\n" + "="*60)
        print("ПРИМЕНЕНИЕ ПРЕОБРАБОТКИ К НОВЫМ ДАННЫМ")
        print("="*60)
        
        df_processed = df.copy()
        
        # Сохраняем ID
        if 'id' in df_processed.columns:
            ids = df_processed['id'].copy()
        else:
            ids = None
        
        # Сохраняем target если есть
        if target_col and target_col in df_processed.columns:
            y = df_processed[target_col].copy()
        else:
            y = None
        
        print(f"Начальный размер: {df_processed.shape}")
        
        # 1. Удаление бесполезных признаков
        cols_to_drop = [col for col in self.useless_cols + self.constant_cols 
                       if col in df_processed.columns]
        df_processed = df_processed.drop(columns=cols_to_drop, errors='ignore')
        
        # 2. Обработка дат
        if 'дата_первого_займа' in df_processed.columns:
            df_processed['дата_первого_займа'] = pd.to_datetime(
                df_processed['дата_первого_займа'], format='%m-%Y', errors='coerce'
            )
            current_date = pd.Timestamp('2026-01-01')
            df_processed['стаж_кредитной_истории_мес'] = (
                (current_date - df_processed['дата_первого_займа']).dt.days / 30
            ).fillna(0)
            df_processed['стаж_кредитной_истории_мес'] = df_processed['стаж_кредитной_истории_мес'].clip(0, 600)
            df_processed = df_processed.drop(columns=['дата_первого_займа'])
        
        # 3. Обработка сроков займа
        if 'срок_займа' in df_processed.columns:
            df_processed['срок_займа_мес'] = df_processed['срок_займа'].str.extract('(\d+)').astype(float)
            df_processed['срок_займа_мес'] = df_processed['срок_займа_мес'] * 12
            df_processed = df_processed.drop(columns=['срок_займа'])
        
        # 4. Кодирование рейтингов
        if 'рейтинг' in df_processed.columns:
            df_processed['рейтинг_encoded'] = df_processed['рейтинг'].map(self.rating_order).fillna(0)
            df_processed = df_processed.drop(columns=['рейтинг'])
        
        # 5. Стаж работы
        if 'стаж' in df_processed.columns:
            df_processed['стаж_encoded'] = df_processed['стаж'].map(self.experience_order)
            if 'стаж_encoded' in self.medians_:
                df_processed['стаж_encoded'] = df_processed['стаж_encoded'].fillna(self.medians_['стаж_encoded'])
            df_processed = df_processed.drop(columns=['стаж'])
        
        # 6. Бинарные признаки
        for col in self.binary_cols:
            if col in df_processed.columns:
                df_processed[col] = df_processed[col].map(self.binary_mapping).fillna(0)
        
        # 7. One-Hot Encoding
        for col in self.onehot_cols:
            if col in df_processed.columns:
                # Заполняем пропуски
                df_processed[col] = df_processed[col].fillna('MISSING')
                
                # Создаем dummy-переменные только для сохраненных категорий
                if col in self.onehot_categories_:
                    for category in self.onehot_categories_[col]:
                        df_processed[f'{col}_{category}'] = (df_processed[col] == category).astype(int)
                
                df_processed = df_processed.drop(columns=[col])
        
        # 8. Frequency Encoding
        for col in self.freq_cols:
            if col in df_processed.columns:
                df_processed[col] = df_processed[col].fillna('MISSING')
                if col in self.freq_encoders_:
                    df_processed[f'{col}_freq_encoded'] = df_processed[col].map(self.freq_encoders_[col])
                    # Заполняем пропуски средним значением частоты
                    mean_freq = self.freq_encoders_[col].mean()
                    df_processed[f'{col}_freq_encoded'] = df_processed[f'{col}_freq_encoded'].fillna(mean_freq)
                df_processed = df_processed.drop(columns=[col])
        
        # 9. Цель займа (группировка)
        if 'цель_займа' in df_processed.columns:
            purpose_groups = {
                'консолидация_долга': ['консолидация_долга'],
                'кредитная_карта': ['кредитная_карта'],
                'жилье': ['улучшение_жилищных_условий', 'дом'],
                'бизнес': ['мелкий_бизнес'],
                'авто': ['автомобиль'],
                'образование': ['образование'],
                'лечение': ['лечение'],
                'переезд': ['переезд'],
                'отпуск': ['отпуск'],
                'другое': ['другое', 'крупная_покупка', 'возобновляемая_энергия', 'свадьба']
            }
            
            purpose_to_group = {}
            for group, purposes in purpose_groups.items():
                for purpose in purposes:
                    purpose_to_group[purpose] = group
            
            df_processed['цель_займа'] = df_processed['цель_займа'].fillna('другое')
            df_processed['цель_займа_группа'] = df_processed['цель_займа'].map(purpose_to_group)
            df_processed.loc[df_processed['цель_займа_группа'].isna(), 'цель_займа_группа'] = 'другое'
            
            # One-hot для сгруппированной цели
            if 'цель_займа_группа' in self.onehot_categories_:
                for category in self.onehot_categories_['цель_займа_группа']:
                    df_processed[f'цель_займа_{category}'] = (df_processed['цель_займа_группа'] == category).astype(int)
            
            df_processed = df_processed.drop(columns=['цель_займа', 'цель_займа_группа'])
        
        # 10. Обработка пени_за_дефолт
        if 'пени_за_дефолт' in df_processed.columns:
            df_processed['пени_за_дефолт'] = df_processed['пени_за_дефолт'].map({'True': 1, 'False': 0})
            if 'пени_за_дефолт' in self.medians_:
                df_processed['пени_за_дефолт'] = df_processed['пени_за_дефолт'].fillna(self.medians_['пени_за_дефолт'])
        
        # 11. Финансовые соотношения
        if all(col in df_processed.columns for col in ['аннуитет', 'годовой_доход']):
            df_processed['годовой_доход_safe'] = df_processed['годовой_доход'].replace(0, 1)
            df_processed['аннуитет_к_доходу'] = df_processed['аннуитет'] * 12 / df_processed['годовой_доход_safe']
            df_processed = df_processed.drop(columns=['годовой_доход_safe'])
        
        if all(col in df_processed.columns for col in ['пдн', 'годовой_доход']):
            df_processed['годовой_доход_safe'] = df_processed['годовой_доход'].replace(0, 1)
            df_processed['пдн_от_дохода'] = df_processed['пдн'] / df_processed['годовой_доход_safe']
            df_processed = df_processed.drop(columns=['годовой_доход_safe'])
        
        if all(col in df_processed.columns for col in ['сумма_займа', 'годовой_доход']):
            df_processed['годовой_доход_safe'] = df_processed['годовой_доход'].replace(0, 1)
            df_processed['заем_к_доходу'] = df_processed['сумма_займа'] / df_processed['годовой_доход_safe']
            df_processed = df_processed.drop(columns=['годовой_доход_safe'])
        
        # 12. Заполнение пропусков в числовых признаках
        for col, median_val in self.medians_.items():
            if col in df_processed.columns:
                df_processed[col] = df_processed[col].fillna(median_val)
        
        # 13. Масштабирование
        if self.scaler_ is not None:
            numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
            if numeric_cols:
                df_processed[numeric_cols] = self.scaler_.transform(df_processed[numeric_cols])
        
        # 14. Выравнивание колонок с train
        if self.feature_names_ is not None:
            # Добавляем отсутствующие колонки
            for col in self.feature_names_:
                if col not in df_processed.columns:
                    df_processed[col] = 0
            
            # Удаляем лишние колонки
            extra_cols = [col for col in df_processed.columns if col not in self.feature_names_]
            if extra_cols:
                df_processed = df_processed.drop(columns=extra_cols)
            
            # Упорядочиваем колонки как в train
            df_processed = df_processed[self.feature_names_]
        
        print(f"Финальный размер: {df_processed.shape}")
        print("="*60)
        
        return df_processed, y, ids
    
    def save(self, path: str):
        """Сохраняет препроцессор на диск"""
        joblib.dump(self, path)
        print(f"Препроцессор сохранен в {path}")
    
    @classmethod
    def load(cls, path: str):
        """Загружает препроцессор с диска"""
        return joblib.load(path)

<>:94: SyntaxWarning: invalid escape sequence '\d'
<>:263: SyntaxWarning: invalid escape sequence '\d'
<>:94: SyntaxWarning: invalid escape sequence '\d'
<>:263: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_977230/1746791803.py:94: SyntaxWarning: invalid escape sequence '\d'
  train_df_processed['срок_займа_мес'] = train_df_processed['срок_займа'].str.extract('(\d+)').astype(float)
/tmp/ipykernel_977230/1746791803.py:263: SyntaxWarning: invalid escape sequence '\d'
  df_processed['срок_займа_мес'] = df_processed['срок_займа'].str.extract('(\d+)').astype(float)


In [6]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

# 1. Загружаем ИСХОДНЫЕ данные
train = pd.read_csv('../data/shift_ml_2026_train.csv')
test = pd.read_csv('../data/shift_ml_2026_test.csv')

# 2. Создаем и обучаем препроцессор на ВСЕХ train данных
preprocessor = CreditScoringPreprocessor(RANDOM_STATE=42)

# Обучаем препроцессор на train данных
preprocessor.fit(train, target_col='итоговый_статус_займа')

# Получаем обработанные train данные
X_train_all, y_train_all, train_ids_all = preprocessor.transform(train, target_col='итоговый_статус_займа')

# 3. Разделяем на train/val для валидации (80/20)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_all, y_train_all, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train_all
)

print(f"Размеры данных:")
print(f"  X_train_split: {X_train_split.shape}")
print(f"  X_val_split:   {X_val_split.shape}")
print(f"  y_train_split: {y_train_split.shape}")
print(f"  y_val_split:   {y_val_split.shape}")

# 4. Обучаем модель на train_split
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 7,
    'num_leaves': 31,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
}

lgb_model = lgb.LGBMClassifier(**lgb_params)
lgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val_split, y_val_split)],
    eval_metric='auc',
    early_stopping_rounds=50,
    verbose=100
)

# 5. Оцениваем на val (для проверки)
val_predictions = lgb_model.predict_proba(X_val_split)[:, 1]
val_auc = roc_auc_score(y_val_split, val_predictions)
print(f"\nVal AUC: {val_auc:.4f}")

# 6. Переобучаем на ВСЕХ train данных для финального сабмита
final_model = lgb.LGBMClassifier(**lgb_params)
final_model.fit(X_train_all, y_train_all)

print(f"\nФинальная модель обучена на {len(X_train_all)} записях")

# 7. Применяем препроцессор к ИСХОДНОМУ test
X_test_processed, _, test_ids = preprocessor.transform(test)

print(f"\nTest данные обработаны:")
print(f"  X_test_processed: {X_test_processed.shape}")
print(f"  test_ids: {len(test_ids)}")

# Проверяем, что признаки совпадают
print(f"\nПроверка совпадения признаков:")
print(f"  Train features: {len(X_train_all.columns)}")
print(f"  Test features:  {len(X_test_processed.columns)}")

if list(X_train_all.columns) == list(X_test_processed.columns):
    print("✓ Признаки совпадают!")
else:
    print("⚠ Признаки не совпадают!")
    # Находим различия
    train_cols = set(X_train_all.columns)
    test_cols = set(X_test_processed.columns)
    print(f"  В train, но нет в test: {train_cols - test_cols}")
    print(f"  В test, но нет в train: {test_cols - train_cols}")

# 8. Предсказания на тесте
test_predictions = final_model.predict_proba(X_test_processed)[:, 1]

print(f"\nСтатистика предсказаний на тесте:")
print(f"  Min:  {test_predictions.min():.6f}")
print(f"  Max:  {test_predictions.max():.6f}")
print(f"  Mean: {test_predictions.mean():.6f}")
print(f"  Std:  {test_predictions.std():.6f}")

# Для сравнения - предсказания на train
train_predictions = final_model.predict_proba(X_train_all)[:, 1]
print(f"\nСтатистика предсказаний на train:")
print(f"  Min:  {train_predictions.min():.6f}")
print(f"  Max:  {train_predictions.max():.6f}")
print(f"  Mean: {train_predictions.mean():.6f}")
print(f"  Std:  {train_predictions.std():.6f}")

# 9. Создаем сабмит
submission = pd.DataFrame({
    'ID': test_ids,
    'Proba': test_predictions
})

print("\n" + "="*60)
print("ПРОВЕРКА САБМИТА:")
print("="*60)
print(f"Размер: {submission.shape}")
print(f"Колонки: {submission.columns.tolist()}")
print(f"Типы данных:\n{submission.dtypes}")

print("\nПервые 10 строк сабмита:")
print(submission.head(10).to_string(index=False))

# Проверка на NaN
if submission['Proba'].isna().any():
    nan_count = submission['Proba'].isna().sum()
    print(f"\n⚠ ВНИМАНИЕ: Найдено {nan_count} NaN значений!")
    # Заполняем средним
    mean_val = submission['Proba'].mean()
    submission['Proba'] = submission['Proba'].fillna(mean_val)
    print(f"Заполнено средним значением: {mean_val:.6f}")

# Сохраняем сабмит
submission.to_csv('final_submission_improved.csv', index=False)
print(f"\n✅ Сабмит сохранен: final_submission_improved.csv")

/tmp/ipykernel_977230/2784789786.py:10: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/shift_ml_2026_train.csv')


ОБУЧЕНИЕ ПРЕПРОЦЕССОРА
Начальный размер: (1210779, 109)
✓ Удалено 7 признаков
✓ Создан признак: стаж_кредитной_истории_мес
✓ Создан признак: срок_займа_мес
✓ Закодирован рейтинг
✓ Закодирован стаж
✓ Закодирован пос_стоп_фактор
✓ Закодирован юридический_статус
✓ Сохранены категории для владение_жильем: 6 значений
✓ Сохранены категории для подтвержден_ли_доход: 3 значений
✓ Сохранены категории для первоначальный_статус_займа: 2 значений
✓ Сохранены категории для тип_займа: 2 значений
✓ Сохранены категории для тип_предоставления_кредита: 2 значений
✓ Сохранены частоты для профессия_заемщика
✓ Сохранены частоты для допрейтинг
✓ Сохранены частоты для регион
✓ Обработана цель_займа
✓ Обработано пени_за_дефолт


/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


✓ Создан: аннуитет_к_доходу
✓ Создан: пдн_от_дохода
✓ Создан: заем_к_доходу


/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1217: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,
/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1409: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


✓ Обучен scaler на 96 признаках
✓ Удалено 3 константных признаков

ПРЕПРОЦЕССОР ОБУЧЕН!
Количество признаков: 103

ПРИМЕНЕНИЕ ПРЕОБРАБОТКИ К НОВЫМ ДАННЫМ
Начальный размер: (1210779, 109)


/tmp/ipykernel_977230/1746791803.py:94: SyntaxWarning: invalid escape sequence '\d'
  train_df_processed['срок_займа_мес'] = train_df_processed['срок_займа'].str.extract('(\d+)').astype(float)
/tmp/ipykernel_977230/1746791803.py:263: SyntaxWarning: invalid escape sequence '\d'
  df_processed['срок_займа_мес'] = df_processed['срок_займа'].str.extract('(\d+)').astype(float)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- владение_жильем_АРЕНДА
- владение_жильем_ДРУГОЕ
- владение_жильем_ИПОТЕКА
- владение_жильем_ЛЮБОЕ
- владение_жильем_НЕТ
- ...


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Кросс-валидация
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(
    lgb_model, X_train_all, y_train_all,
    cv=cv, scoring='roc_auc', n_jobs=-1
)

print(f"CV AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")